In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # 使用cpu训练

In [5]:
def get_uncompiled_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = layers.Dense(10, activation="softmax", name="predictions")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

In [6]:
class CategoricalTruePositives(keras.metrics.Metric):
    """
    自定义评估函数
    继承自:tf.keras.metrics.Mtric
    必须实现__init__、update_state、result、reset_state方法
    """

    def __init__(self, name="categorical_true_positives", **kwargs):
        """__init__(self), in which you will create state variables for your metric."""
        super(CategoricalTruePositives, self).__init__(name=name, **kwargs)  # ★★★★★不能省略

        self.true_positives = self.add_weight(name="ctp", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        """update_state(self, y_true, y_pred, sample_weight=None), which uses the targets y_true and the model predictions y_pred to update the state variables."""
        y_pred = tf.reshape(tf.argmax(y_pred, axis=1), shape=(-1, 1))
        values = tf.cast(y_true, "int32") == tf.cast(y_pred, "int32")
        values = tf.cast(values, "float32")
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, "float32")
            values = tf.multiply(values, sample_weight)
        self.true_positives.assign_add(tf.reduce_sum(values))

    def result(self):
        """result(self), which uses the state variables to compute the final results."""
        return self.true_positives

    def reset_state(self):
        """reset_state(self), which reinitializes the state of the metric."""
        self.true_positives.assign(0.0)  # The state of the metric will be reset at the start of each epoch.


model = get_uncompiled_model()
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[CategoricalTruePositives()],
)
model.fit(x_train, y_train, batch_size=64, epochs=5)

Epoch 1/5
782/782 [==============================] - 2s 1ms/step - loss: 0.3434 - categorical_true_positives: 45170.0000